In [1]:
import numpy as np
import os

# Load weights

In [2]:
weights_folder = os.path.join('.','weights')

weightsI1 = np.load(os.path.join(weights_folder, 'weightsI1.npy'))
bias1     = np.load(os.path.join(weights_folder, 'bias1.npy'))
weights12 = np.load(os.path.join(weights_folder, 'weights12.npy'))
bias2     = np.load(os.path.join(weights_folder, 'bias2.npy'))
weights23 = np.load(os.path.join(weights_folder, 'weights23.npy'))
bias3     = np.load(os.path.join(weights_folder, 'bias3.npy'))
weights3O = np.load(os.path.join(weights_folder, 'weights3O.npy'))
biasO     = np.load(os.path.join(weights_folder, 'biasO.npy'))

# Define nn model

In [3]:
def nn_model(inputs, 
             weightsI1, bias1,
             weights12, bias2,
             weights23, bias3,
             weights3O, biasO,
             SCALE = 15,
             int_type=np.int32):
    
    # # Scale the inputs by 2^SCALE
    # inputs = (inputs * (2 ** SCALE)).astype(int_type)
    # Layer 1
    hidden1 = np.zeros(100, dtype=int_type)
    for h1 in range(100):
        sum_ = bias1[h1]
        for i in range(12):
            sum_ += (inputs[i] * weightsI1[i][h1]) >> SCALE
            # print(bin(sum_))
        hidden1[h1] = max(0, sum_)  # ReLU activation

    # Layer 2
    hidden2 = np.zeros(100, dtype=int_type)
    for h2 in range(100):
        sum_ = bias2[h2]
        for h1 in range(100):
            sum_ += (hidden1[h1] * weights12[h1][h2]) >> SCALE
        hidden2[h2] = max(0, sum_)  # ReLU activation

    # Layer 3
    hidden3 = np.zeros(100, dtype=int_type)
    for h3 in range(100):
        sum_ = bias3[h3]
        for h2 in range(100):
            sum_ += (hidden2[h2] * weights23[h2][h3]) >> SCALE
        hidden3[h3] = max(0, sum_)  # ReLU activation

    # Output layer
    outputs = np.zeros(2, dtype=int_type)
    for o in range(2):
        sum_ = biasO[o]
        for h3 in range(100):
            sum_ += (hidden3[h3] * weights3O[h3][o]) >> SCALE
        outputs[o] = sum_

    return outputs

# Example

In [4]:
example_folder = os.path.join('.','example_data')

In [5]:
# Read nn input
nn_input = np.load(os.path.join(example_folder, 'nn_input.npy'))
# Flare location obtained from imaging. Just used to compare the results of the quantized nn
# with the actual flare location
flare_loc_im = np.load(os.path.join(example_folder, 'flare_loc_im.npy'))

In [6]:
# Normalize and scale data
SCALE = 15

max_nn_input    = np.max(nn_input)
nn_input_scaled = (nn_input * (2 ** SCALE)) // max_nn_input

In [7]:
# NN prediction
nn_out = nn_model(nn_input_scaled, 
                  weightsI1, bias1,
                  weights12, bias2,
                  weights23, bias3,
                  weights3O, biasO,
                  SCALE = SCALE)

In [8]:
# Scale nn prediction
xy_flare_norm = 4000
nn_out = (nn_out * xy_flare_norm) >> SCALE

In [9]:
# Compare with flare loc from imaging
print("Flare location from imaging")
print("X: "+str(flare_loc_im[0]))
print("Y: "+str(flare_loc_im[1]))
print()
print("NN prediction")
print("X: "+str(nn_out[0]))
print("Y: "+str(nn_out[1]))

Flare location from imaging
X: -272.0
Y: -16.0

NN prediction
X: -324
Y: -10
